# Loading the OHHLA corpus
This notebook shows a typical example of data loading and preprocessing necessary for NLP. In this case we are loading a corpus downloaded from the Hip-Hop Lyrics webpage [www.ohhla.com](www.ohhla.com). Our primary goal is to provide a dataset loading function for the [language modelling](todo) chapter in this book.

We provide the corpus in the `data` directory. As this notebook lives in a sub-directory itself, we access it via `../data`. Before preprocessing all files and provide *generic* loaders it is useful to inspect the format of the files based on a specific example file, and work on the loading process in this context. Here we look at `/data/ohhla/www.ohhla.com/anonymous/j_live/SPTA/authentc.jlv.txt`.  

In [42]:
with open('../data/ohhla/www.ohhla.com/anonymous/j_live/allabove/satisfy.jlv.txt.html', 'r') as f:
    # we use read().splitlines() instead of readlines() to skip newline characters
    lines = f.read().splitlines()
    
lines

['<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">',
 '<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en">',
 '',
 '<head>',
 '        <meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" />',
 '        <meta http-equiv="charset" content="ISO-8859-1" />',
 '        <meta http-equiv="content-language" content="English" />',
 '        <meta http-equiv="vw96.object type" content="Document" />',
 '        <meta name="resource-type" content="document" />',
 '        <meta name="distribution" content="Global" />',
 '        <meta name="rating" content="General" />',
 '        <meta name="robots" content="all" />',
 '        <meta name="revist-after" content="2 days" />',
 '        <link rel="shortcut icon" href="../../../favicon.ico" />',
 '        <title>The Original Hip-Hop (Rap) Lyrics Archive</title>',
 '',
 '        <!-- link rel="stylesheet" type="text/css" href="http://ohhla.com/files/main.css" / -->',
 '\t     

We first would like to remove everything outside of the `<pre>` tag, and then remove the meta information.

In [56]:
def find_lyrics(lines):
    filtered = []
    in_pre = False
    for line in lines:
        if '<pre>' in line:
            in_pre = True
            filtered.append(line.replace("<pre>",""))
        elif '</pre>' in line:
            in_pre = False
            filtered.append(line.replace("</pre>",""))
        elif in_pre:
            filtered.append(line)
    return filtered[6:]
    
lyrics = find_lyrics(lines)
lyrics

['Hey yo',
 'Lights, camera, tragedy, comedy, romance',
 'You better dance from your fighting stance',
 "Or you'll never have a fighting chance",
 'In the rat race',
 "Where the referee's son started way in advance",
 "But still you livin' the American Dream",
 "Silk PJ's, sheets and down pillows",
 'Who the fuck would wanna wake up?',
 'You got it good like hot sex after the break up',
 "Your four car garage it's just more space to take up",
 'You even bought your mom a new whip scrap the jalopy',
 'Thousand dollar habit, million dollar hobby',
 'You a success story everybody wanna copy',
 'But few work for it, most get jerked for it',
 "If you think that you could ignore it, you're ig-norant",
 'A fat wallet still never made a man free',
 'They say to eat good, yo, you gotta swallow your pride',
 "But dead that game plan, I'm not satisfied",
 '',
 '[Chorus]',
 'The poor get worked, the rich get richer',
 'The world gets worse, do you get the picture?',
 'The poor gets dead, the rich 

Finally, we would like to convert the list of lines with newline characters to a single string, as this will be easier to process for our language models. We will also mark lyrical "bars" (lines) using a `BAR` tag to still capture the rhythmical structure in the song.

In [57]:
string = '[BAR]' + '[/BAR][BAR]'.join(lyrics) + '[/BAR]'
string

"[BAR]Hey yo[/BAR][BAR]Lights, camera, tragedy, comedy, romance[/BAR][BAR]You better dance from your fighting stance[/BAR][BAR]Or you'll never have a fighting chance[/BAR][BAR]In the rat race[/BAR][BAR]Where the referee's son started way in advance[/BAR][BAR]But still you livin' the American Dream[/BAR][BAR]Silk PJ's, sheets and down pillows[/BAR][BAR]Who the fuck would wanna wake up?[/BAR][BAR]You got it good like hot sex after the break up[/BAR][BAR]Your four car garage it's just more space to take up[/BAR][BAR]You even bought your mom a new whip scrap the jalopy[/BAR][BAR]Thousand dollar habit, million dollar hobby[/BAR][BAR]You a success story everybody wanna copy[/BAR][BAR]But few work for it, most get jerked for it[/BAR][BAR]If you think that you could ignore it, you're ig-norant[/BAR][BAR]A fat wallet still never made a man free[/BAR][BAR]They say to eat good, yo, you gotta swallow your pride[/BAR][BAR]But dead that game plan, I'm not satisfied[/BAR][BAR][/BAR][BAR][Chorus][/BAR

We are now ready to provide a loading function. 

In [60]:
def load_song(file_name):
    def load_raw(encoding):
        with open(file_name, 'r',encoding=encoding) as f:
            # we use read().splitlines() instead of readlines() to skip newline characters
            lines = f.read().splitlines()   
            # some files are pure txt files for which we don't need to extract the lyrics 
            lyrics = find_lyrics(lines) if file_name.endswith('html') else lines[5:]
            string = '[BAR]' + '[/BAR][BAR]'.join(lyrics) + '[/BAR]'
            return string
    try:
        return load_raw('utf-8')
    except UnicodeDecodeError:
        return load_raw('cp1252')
        
    
load_song('../data/ohhla/www.ohhla.com/anonymous/j_live/allabove/satisfy.jlv.txt.html')

"[BAR]Hey yo[/BAR][BAR]Lights, camera, tragedy, comedy, romance[/BAR][BAR]You better dance from your fighting stance[/BAR][BAR]Or you'll never have a fighting chance[/BAR][BAR]In the rat race[/BAR][BAR]Where the referee's son started way in advance[/BAR][BAR]But still you livin' the American Dream[/BAR][BAR]Silk PJ's, sheets and down pillows[/BAR][BAR]Who the fuck would wanna wake up?[/BAR][BAR]You got it good like hot sex after the break up[/BAR][BAR]Your four car garage it's just more space to take up[/BAR][BAR]You even bought your mom a new whip scrap the jalopy[/BAR][BAR]Thousand dollar habit, million dollar hobby[/BAR][BAR]You a success story everybody wanna copy[/BAR][BAR]But few work for it, most get jerked for it[/BAR][BAR]If you think that you could ignore it, you're ig-norant[/BAR][BAR]A fat wallet still never made a man free[/BAR][BAR]They say to eat good, yo, you gotta swallow your pride[/BAR][BAR]But dead that game plan, I'm not satisfied[/BAR][BAR][/BAR][BAR][Chorus][/BAR

Now we want to load several files from an album directory. 

In [61]:
from os import listdir
from os.path import isfile, join

def load_album(path):
    # we filter out directories, and files that don't look like song files in OHHLA.
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f)) and 'txt' in f]
    lyrics = [load_song(f) for f in onlyfiles]
    return lyrics

songs = load_album('../data/ohhla/www.ohhla.com/anonymous/j_live/SPTA/')
[len(s) for s in songs]

[2555, 2779, 3283]

We will also make it easy to load several albums. Then, for a few artists we provide short cuts to the album directories we care about. 

In [63]:
def load_albums(album_paths):
    return [song 
            for path in album_paths 
            for song in load_album(path)]

top_dir = '../data/ohhla/www.ohhla.com/anonymous/'
j_live = [
    top_dir + '/j_live/allabove/',
    top_dir + '/j_live/bestpart/'
]
len(load_albums(j_live))

29

It will be useful to convert a list of documents into a flat list of tokens. Based on the approach showed in the [tokenisation chapter](todo) we can do this as follows:

In [74]:
import re
token = re.compile('\'|\[BAR\]|\[/BAR\]|[\w-]+')
def words(docs):
    return [word 
            for doc in docs 
            for word in token.findall(doc)]
words(songs)

['[BAR]',
 'J-Live',
 '[/BAR]',
 '[BAR]',
 'Well',
 'if',
 'isn',
 "'",
 't',
 'the',
 'outbreak',
 'monkey',
 'for',
 'that',
 'latest',
 'epidemic',
 'of',
 'The',
 'Vapors',
 '[/BAR]',
 '[BAR]',
 'Bringing',
 'you',
 'the',
 'greatest',
 'live',
 'caper',
 'to',
 'date',
 '[/BAR]',
 '[BAR]',
 'To',
 'get',
 'involved',
 'to',
 'marry',
 'to',
 'the',
 'music',
 '[/BAR]',
 '[BAR]',
 'Givin',
 'no',
 'quarter',
 'for',
 'those',
 'that',
 'would',
 'abuse',
 'it',
 '[/BAR]',
 '[BAR]',
 '[/BAR]',
 '[BAR]',
 'Nah',
 'not',
 'even',
 'a',
 'fuck',
 'and',
 'a',
 'dream',
 'a',
 'Coke',
 'and',
 'smile',
 '[/BAR]',
 '[BAR]',
 'A',
 'smoke',
 'and',
 'a',
 'pancake',
 'not',
 'a',
 'pat',
 'on',
 'the',
 'back',
 'and',
 'a',
 'handshake',
 '[/BAR]',
 '[BAR]',
 'For',
 'bein',
 'whack',
 'shit',
 'is',
 'beyond',
 'hate',
 '[/BAR]',
 '[BAR]',
 'I',
 "'",
 'm',
 'evacuatin',
 'the',
 'state',
 'you',
 'makin',
 'me',
 'glands',
 'ache',
 '[/BAR]',
 '[BAR]',
 '[/BAR]',
 '[BAR]',
 'Me',
 'and